In [7]:
import numpy as np
import pandas as pd
import os

In [6]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.

In [12]:
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_2_train.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X=list(dataset2_filtered['text'])
y=list(dataset2_filtered['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [ ]:
dataset1=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_train.csv")
X_train=list(dataset1['text'])
y_train=list(dataset1['label'])
#dataset1.shape
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_test.csv")
X_test=list(dataset2['text'])
y_test=list(dataset2['label'])
#dataset2.shape
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [13]:
train_df.shape

(29493, 2)

In [14]:
test_df.shape

(12640, 2)

In [3]:
import torch
torch.cuda.device_count()

1

In [4]:
device = torch.cuda.get_device_properties(0)  # Assuming you have at least one GPU
print(device)

_CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16151MB, multi_processor_count=80)


In [15]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

# Tamil_BERT

In [18]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 4,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'train_batch_size': 64,
    "learning_rate":1e-5,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

tbert_model_tamil = ClassificationModel(
    "bert",
    "l3cube-pune/tamil-bert",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/tamil-bert and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
tbert_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.2841047294328556)

In [22]:
# Evaluate the model
result, model_outputs, wrong_predictions = tbert_model_tamil.eval_model(test_df)

  0%|          | 0/25 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/127 [00:00<?, ?it/s]

In [23]:
print(result) #result of the model

{'mcc': 0.5493337566853059, 'accuracy': 0.8383702531645569, 'f1_score': 0.6539047941724547, 'tp': 1930, 'tn': 8667, 'fp': 910, 'fn': 1133, 'auroc': 0.8776085416036645, 'auprc': 0.686999574819202, 'eval_loss': 0.4751479256106174}


In [20]:
predictions, raw_outputs = tbert_model_tamil.predict(list(test_df['text'].values))

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

In [21]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      9800
           1       0.63      0.68      0.65      2840

    accuracy                           0.84     12640
   macro avg       0.77      0.78      0.77     12640
weighted avg       0.84      0.84      0.84     12640



In [24]:
import pickle
# save the model to disk
filename = 'tbert_tamil.sav'
pickle.dump(tbert_model_tamil, open(filename, 'wb'))

In [25]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
sample_text="Ethu padama ah ila serial ah"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="Ethu padame illa, pakka serial. mokka padam"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="அந்த உணவகத்தில் உணவும் நன்றாக இருந்தது"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0])

In [26]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8383702531645569


In [27]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8362473343765485


In [28]:
import shutil

local_model_path = '/content/sample_data/tbert_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/tbert_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/tbert_tamil.sav'

In [ ]:
#CATBOOST